In [1]:
from multiscaleio.datasets.get_arpa import get_arpa
from multiscaleio.core.multiscaling import ReshiftedExpansion
import pandas as pd

c:\Users\Tobia\anaconda3\envs\multiscaleio\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
c:\Users\Tobia\anaconda3\envs\multiscaleio\lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
arpa21 = get_arpa(n_jobs=5)

In [3]:
rs = ReshiftedExpansion(window=2, date_col="data", keep_t0=True)
data = rs.fit_transform(arpa21.drop(["lng", "lat", "provincia"], axis=1))

In [4]:
pd.DataFrame(data, columns=rs.get_feature_names_out())

,Ammoniaca,Benzene,Biossido di Azoto,Biossido di Zolfo,BlackCarbon,Monossido di Azoto,Monossido di Carbonio,Ossidi di Azoto,Ozono,PM10 (SM2005),...,Benzene_shift_1,Biossido di Azoto_shift_1,Biossido di Zolfo_shift_1,BlackCarbon_shift_1,Monossido di Azoto_shift_1,Monossido di Carbonio_shift_1,Ossidi di Azoto_shift_1,Ozono_shift_1,PM10 (SM2005)_shift_1,Particelle sospese PM2.5_shift_1
0,2021-01-01,NaN,1.433333,28.519444,2.939583,NaN,NaN,0.763333,42.826389,23.928125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,NaN,1.0,28.232407,3.135417,NaN,NaN,0.665833,42.732407,30.742708,...,1.433333,28.519444,2.939583,NaN,NaN,0.763333,42.826389,23.928125,43.0,36.4
2,2021-01-03,NaN,0.633333,18.002083,2.936458,NaN,NaN,0.581667,24.264236,44.576042,...,1.0,28.232407,3.135417,NaN,NaN,0.665833,42.732407,30.742708,14.875,12.0
3,2021-01-04,NaN,1.033333,29.554282,3.445833,NaN,NaN,0.633333,48.063542,21.542708,...,0.633333,18.002083,2.936458,NaN,NaN,0.581667,24.264236,44.576042,10.375,8.4
4,2021-01-05,NaN,1.1,26.997454,3.465625,NaN,NaN,0.601667,44.580093,22.245833,...,1.033333,29.554282,3.445833,NaN,NaN,0.633333,48.063542,21.542708,14.0,10.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,2021-12-27,NaN,NaN,35.101667,4.589583,NaN,NaN,1.141667,111.6575,3.821875,...,NaN,34.313333,4.125,NaN,NaN,1.056944,94.790625,6.454167,60.25,53.5
4376,2021-12-28,NaN,NaN,31.29,4.204167,NaN,NaN,1.015278,100.0075,4.745833,...,NaN,35.101667,4.589583,NaN,NaN,1.141667,111.6575,3.821875,55.0,46.0
4377,2021-12-29,NaN,NaN,28.5925,3.7375,NaN,NaN,0.715278,71.929167,12.805208,...,NaN,31.29,4.204167,NaN,NaN,1.015278,100.0075,4.745833,54.5,47.0
4378,2021-12-30,NaN,NaN,37.4475,3.772917,NaN,NaN,0.890278,129.085833,4.659028,...,NaN,28.5925,3.7375,NaN,NaN,0.715278,71.929167,12.805208,34.5,23.5
